In [1]:
%%capture
## compile PyRoss for this notebook
import os
owd = os.getcwd()
os.chdir('../')
%run setup.py install
os.chdir(owd)

In [2]:

%matplotlib inline
import numpy as np
import pyross
import pandas as pd
import matplotlib.pyplot as plt
from scipy.io import loadmat
from scipy import optimize


In [3]:
M=16  # number of age groups

# load age structure data
my_data = np.genfromtxt('data/age_structures/India-2019.csv', delimiter=',', skip_header=1)
aM, aF = my_data[:, 1], my_data[:, 2]

# set age groups
Ni=aM+aF;   Ni=Ni[0:M];  N=np.sum(Ni)

In [4]:

# contact matrices
my_data = pd.read_excel('data/contact_matrices_152_countries/MUestimates_home_1.xlsx', sheet_name='India',index_col=None)
CH = np.array(my_data)

my_data = pd.read_excel('data/contact_matrices_152_countries/MUestimates_work_1.xlsx', sheet_name='India',index_col=None)
CW = np.array(my_data)

my_data = pd.read_excel('data/contact_matrices_152_countries/MUestimates_school_1.xlsx', sheet_name='India',index_col=None)
CS = np.array(my_data)

my_data = pd.read_excel('data/contact_matrices_152_countries/MUestimates_other_locations_1.xlsx', sheet_name='India',index_col=None)
CO = np.array(my_data)

my_data = pd.read_excel('data/contact_matrices_152_countries/MUestimates_all_locations_1.xlsx', sheet_name='India',index_col=None)
CA = np.array(my_data)

# matrix of total contacts
C=CH+CW+CS+CO

In [5]:
def findBeta(x):
    beta=x
    gIa   = 1./7            # recovery rate of asymptomatic infectives 
    gIs   = 1./7            # recovery rate of symptomatic infectives 
    alpha = 0.               # fraction of asymptomatic infectives 
    fsa   = 1                # the self-isolation parameter   


    # initial conditions    
    Is_0 = np.zeros((M));  Is_0[4:11]=4;  Is_0[1:4]=1
    Ia_0 = np.zeros((M))
    R_0  = np.zeros((M))
    S_0  = Ni - (Ia_0 + Is_0 + R_0)
    
    # duration of simulation and data file
    Tf=21;  Nf=21; filename='this.mat'

    # the contact structure is independent of time 
    def contactMatrix(t):
        return C

    # intantiate model
    parameters = {'alpha':alpha,'beta':beta, 'gIa':gIa,'gIs':gIs,'fsa':fsa}
    model = pyross.models.SIR(parameters, M, Ni)


    # run model
    model.simulate(S_0, Ia_0, Is_0, contactMatrix, Tf, Nf, filename)
    
    data=loadmat(filename); t = data['t'][0]; IC  = np.zeros((Nf))
    for i in range(M):
        IC += data['X'][:,2*M+i] 
    
    my_data = np.genfromtxt('data/covid-cases/india.txt', delimiter='', skip_header=7)
    day, cases = my_data[:,0], my_data[:,2]

    error = np.sum(( cases-IC)**2)
    return error

In [6]:
beta0  = 0.0


sol = optimize.root( findBeta, beta0)
sol.x

array([0.01568928])